In [1]:
import pyspark
from delta import *

In [2]:
warehouse_location = 'hdfs://hdfs:9000/hive/warehouse'

In [3]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp").master('spark://master:7077') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", warehouse_location)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b9a1683d-252e-4437-b3fb-54768f091060;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 672ms :: artifacts dl 17ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 fro

In [4]:
spark

In [5]:
path = 'hdfs://hdfs:9000/user/range-temp-table'

In [6]:
data = spark.range(0, 5)
data.write.format('delta').mode('overwrite').save(path)

In [7]:
df = spark.read.format('delta').load(path)
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



In [8]:
data = spark.range(5, 10)
data.write.format('delta').mode('overwrite').save(path)

In [9]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, path)
deltaTable

In [10]:
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

In [11]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|108|
|  9|
|  5|
|106|
+---+



In [12]:
deltaTable.delete(condition = expr("id % 2 == 0"))

In [13]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|  9|
|  5|
+---+



In [14]:
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

In [15]:
deltaTable.toDF().show()

+---+
| id|
+---+
| 13|
| 12|
| 14|
|  9|
| 19|
|  1|
| 18|
|  7|
| 11|
|  2|
| 17|
|  6|
|  0|
| 16|
| 15|
|  4|
|  5|
| 10|
|  8|
|  3|
+---+



In [16]:
deltaTable.toDF().write.format('delta').mode('overwrite').save(path)

In [17]:
deltaTable = DeltaTable.forPath(spark, path)
deltaTable.toDF().show()

+---+
| id|
+---+
| 13|
| 12|
| 14|
|  9|
| 19|
|  1|
| 18|
|  7|
| 11|
|  2|
| 17|
|  6|
|  0|
| 16|
| 15|
|  4|
|  5|
| 10|
|  8|
|  3|
+---+



In [18]:
# !jupyter nbconvert --to script 4.simple-delta-lake.ipynb

In [19]:
# !/opt/spark/bin/spark-submit \
#   --packages io.delta:delta-core_2.12:1.0.0 \
#   --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" \
#   --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog" \
#   --master spark://master:7077 \
#   /home/data/4.simple-delta-lake.py